In [16]:
import time  # import time module
import datetime  # import datetime module (functions)
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [17]:
# Define the ticker, periods, and interval
ticker = 'AMZN'
period1 = int(time.mktime(datetime.datetime(2022, 12, 1, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime.now().timetuple()))
interval = '1d'

In [18]:
# Fetch historical stock data from Yahoo Finance
query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
df = pd.read_csv(query_string)

In [19]:
# Function to convert string date to datetime object
def str_to_datetime(s):
    split = s.split('-')
    year, month, day = int(split[0]), int(split[1]), int(split[2])
    return datetime.datetime(year=year, month=month, day=day)

In [20]:
# Preprocess the data
df['Date'] = df['Date'].apply(str_to_datetime)
df.index = df.pop('Date')
df = df[['Close']]

In [21]:
# Convert dataframe into a supervised learning problem
def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=3):
    first_date = str_to_datetime(first_date_str)
    last_date = str_to_datetime(last_date_str)

    target_date = first_date
    dates = []
    X, Y = [], []

    last_time = False
    while True:
        df_subset = dataframe.loc[:target_date].tail(n + 1)
        if len(df_subset) != n + 1:
            print(f'Error: Window of size {n} is too large for date {target_date}')
            return

        values = df_subset['Close'].to_numpy()
        x, y = values[:-1], values[-1]

        dates.append(target_date)
        X.append(x)
        Y.append(y)

        next_week = dataframe.loc[target_date:target_date + datetime.timedelta(days=7)]
        next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
        next_date_str = next_datetime_str.split('T')[0]
        year, month, day = next_date_str.split('-')
        next_date = datetime.datetime(day=int(day), month=int(month), year=int(year))

        if last_time:
            break

        target_date = next_date

        if target_date == last_date:
            last_time = True

    ret_df = pd.DataFrame({})
    ret_df['Target Date'] = dates
    X = np.array(X)
    for i in range(0, n):
        ret_df[f'Target-{n - i}'] = X[:, i]

    ret_df['Target'] = Y
    return ret_df

In [22]:
# Generate windowed dataframe
dt = df.index[3]  # get the 4th date
curr = str(datetime.datetime.now() - datetime.timedelta(days=2))[:10]  # trimmed the current variable to date
windowed_df = df_to_windowed_df(df, dt, curr, n=3)

AttributeError: 'Timestamp' object has no attribute 'split'

In [ ]:
# Split dataset into features and target
def windowed_df_to_date_X_y(windowed_dataframe):
    df_as_np = windowed_dataframe.to_numpy()
    dates = df_as_np[:, 0]
    middle_matrix = df_as_np[:, 1:-1]
    X = middle_matrix.reshape((len(dates), middle_matrix.shape[1]))
    Y = df_as_np[:, -1]
    return dates, X.astype(np.float32), Y.astype(np.float32)

dates, X, y = windowed_df_to_date_X_y(windowed_df)

In [ ]:
# Split dataset into training, validation, and test sets
q_80 = int(len(dates) * .8)
q_90 = int(len(dates) * .9)
dates_train, X_train, y_train = dates[:q_80], X[:q_80], y[:q_80]
dates_val, X_val, y_val = dates[q_80:q_90], X[q_80:q_90], y[q_80:q_90]
dates_test, X_test, y_test = dates[q_90:], X[q_90:], y[q_90:]

In [ ]:
# Plot the train, validation, and test sets
plt.plot(dates_train, y_train)
plt.plot(dates_val, y_val)
plt.plot(dates_test, y_test)
plt.legend(['Train', 'Validation', 'Test'])
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
# Setup TensorBoard logging directory
log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# Build the model
model = keras.Sequential([
    keras.layers.Input((3, 1)),
    tf.keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])

model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mean_absolute_error'])

# Train the model with TensorBoard callback
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[tensorboard_callback])

In [ ]:
# Make predictions and plot
train_predictions = model.predict(X_train).flatten()
plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.legend(['Training Predictions', 'Training Observations'])
plt.show()

val_predictions = model.predict(X_val).flatten()
plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.legend(['Validation Predictions', 'Validation Observations'])
plt.show()

test_predictions = model.predict(X_test).flatten()
plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.legend(['Testing Predictions', 'Testing Observations'])
plt.show()

In [ ]:
# Launch TensorBoard to visualize the model architecture and training process
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:

# Make predictions for future dates
req = windowed_df.tail(3)
inp = tuple(req['Target'])
npin = np.asarray(inp).reshape(1, -1)
stk_pred = model.predict(npin)
today = float(stk_pred)

req1 = windowed_df.tail(2)
tom = list(req1['Target']) + [today]
npin = np.asarray(tuple(tom)).reshape(1, -1)
tom_pred = model.predict(npin)
tomorrow = float(tom_pred)

In [ ]:
print(f"Predicted closing price for tomorrow: {tomorrow}")